In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = 'true'

import pylab
from matplotlib import pyplot as plt
import numpy as np
import IPython.display as ipd

import scipy.io.wavfile as wav
import wave
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt


data_dir = '/disk2/ofirb/viewmaker/downstream/audioMNIST/free-spoken-digit-dataset-master/recordings'
output_dir =  '/disk2/ofirb/viewmaker/downstream/audioMNIST/output/audio-images'

SIZE = 256
BATCH_SIZE = 32
channels = 3
kernel = 4
stride = 1
pool = 2



# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=output_dir,
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(SIZE, SIZE),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=output_dir,
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(SIZE, SIZE),
                                             subset="validation",
                                             seed=0)





2022-01-14 18:18:03.408783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-14 18:18:03.408814: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Found 3000 files belonging to 10 classes.
Using 2400 files for training.


2022-01-14 18:18:06.303227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-14 18:18:06.303319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-14 18:18:06.303376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-14 18:18:06.303430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-01-14 18:18:06.303484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Found 3000 files belonging to 10 classes.
Using 600 files for validation.


In [2]:
class_names = train_dataset.class_names
num_classes = len(class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = valid_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# model from scratch
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(SIZE, SIZE, channels)),
    layers.Conv2D(16, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Conv2D(32, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Conv2D(64, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 253, 253, 16)      784       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 123, 123, 32)      8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 58, 58, 64)        3

In [5]:
from viewmaker.downstream.downstream_utils import CustomDataGenerator

data_generator = CustomDataGenerator(viewmaker_cpkt=args.ckpt,
                                         config_path=args.config,
                                         horizontal_flip=horizontal_flip,
                                         rotation_range=rotation_range)
data_generator.fit(train_ds)

# Configure the model for training
model.compile(optimizer='adam', 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])





ModuleNotFoundError: No module named 'viewmaker'

In [ ]:
callbacks = [WandbCallback()] if not args.debug else []

epochs = 15

history = model.fit(data_generator.flow(X_train, y_train, batch_size=args.batch_size),
                    callbacks=callbacks,
                    steps_per_epoch=100,
                    epochs=epochs,
                    validation_data=val_ds,
                    verbose=1)